In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath("../"))
import src.utils as utils

In [50]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

## Considerations

There are known to be several approaches on building a RAG system for csv files:
 
- Loading CSV into documents.  

In [ ]:
model_name = "intfloat/multilingual-e5-large-instruct"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

### Load Documents

Splitting into chunks would not be a good idea as the model would loose the context. Another idea would be splitting into chunks but for each chunk storing the name of the recipe, ingredients and authors, basically splitting only the instructions. BAD IDEA. SHOULD I SPLIT OR NOT? 

In [ ]:
loader = CSVLoader(file_path="../data/processed/Recipes_processed_rag.csv")
docs = loader.load()

max_length, max_index = utils.find_max_len_doc(docs)
print(f"Maximum length: {max_length} characters")
print(f"Found in document index: {max_index}")
print("=====================")
print(docs[max_index].page_content)

Maximum length: 1344 characters
Found in document index: 55
id: 67
name: Kuře z jednoho hrnce
author_name: Roman Vaněk
ingredients: Česnek, Chilli koření mleté, Cuketa, Hřebíček mletý, Koriandr mletý, Krájená cibule, Kuřecí vykostěná stehna, Kuřecí vývar, Kurkuma mletá, Med, Mrkev, Olivový olej, Pepř černý , Plocholistá petržel, Pomerančová šťáva, Římský kmín mletý, Rýže, Skořice mletá, Sladká paprika, Sůl, Těstoviny, Víno vhodné k drůbežímu masu, Zázvor
steps: ['Do kastrolu přidáme cuketu a mrkev, promícháme a pečeme 2 minuty. Občas zamícháme. Přidáme cibuli, promícháme a pečeme 1 minutu.', 'Kastrol vložíme do rozehřáté trouby a pečeme 40 minut. Před podáváním přimícháme nakrájenou petržel.', 'Ke koření přilijeme pomerančovou šťávu a vývar, přidáme med a pomerančovou kůru a promícháme. Přivedeme k varu a nalijeme do kastrolu ke kuřeti.', 'Maso vydatně osolíme a opepříme z obou stran.', 'Mrkev nakrájíme na kolečka silná 0,5 cm. Cibuli nakrájíme najemno. Česnek nakrájíme na plátky. Cuke

### Indexing and Storage

As the current code is a test-task, that is designed to run cross-platform, ChromaDB database is used instead of FAISS, as FAISS requires installation based on OS and GPU availability. 

In [ ]:
chroma_db_path = "../data/chroma_db"
os.makedirs(chroma_db_path, exist_ok=True)

doc_texts = [doc.page_content for doc in docs]
doc_embeddings = embeddings.embed_documents(doc_texts)

vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="rag",
    embedding=embeddings,
    db_path=chroma_db_path
)



ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
# always better to be written in english
system_prompt=""